In [1]:
import pandas as pd
import time
import requests
import os
import zipfile
from os import listdir
from os.path import isfile, join
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import metrics

In [10]:
response = requests.get('https://jazz-yt67mbf5eq-ey.a.run.app')
response.status_code

200

In [24]:
url = 'https://jazz-yt67mbf5eq-ey.a.run.app/implement_som'

counter = 1

while counter <= 20:
    # Save Jazz and CSV File for transfer
    files = {
    "jazz_file": open('Kohonen-Maps/Jazz_Files/Sigma/data_dog_domain_file_'+str(counter)+'.jazz', 'rb'), 
    "csv_file": open('Kohonen-Maps/data_dog_prepared.csv','rb')
    }
    # post Request to URL
    response = requests.post(url, files=files)
    # Save the Response ZIP File
    filename = url.split('/')[-1] + '_result.zip'
    with open(os.path.join('Kohonen-Maps', filename), "wb") as output_file:
        output_file.write(response.content)
    # Unzip the File
    with zipfile.ZipFile('Kohonen-Maps/implement_som_result.zip', 'r') as zip_ref:
        name = zip_ref.namelist()
        # extract the "dimension reduced" csv file
        zip_ref.extract(name[2])
    # time.sleep(2.5)
    # Change the File Names in an numerical Order
    onlyfiles = [f for f in listdir('data/processed') if isfile(join('data/processed', f))]
    old_file = os.path.join('data/processed', onlyfiles[0])
    new_file = os.path.join('data/processed', 'v6.'+str(counter)+'_data_dog_KM.csv')
    os.rename(old_file, new_file)
    # Remove the Index Column
    df = pd.read_csv('data/processed/v6.'+str(counter)+'_data_dog_KM.csv')
    df = df.drop('Unnamed: 0', axis=1)
    df.to_csv('data/processed/v6.'+str(counter)+'_data_dog_KM.csv', index=False)

    counter += 1

<h1>Test-Training-Split for V6</h1>

In [25]:
# empty df for the results
df = pd.DataFrame(columns = ['RFC Accuracy', 'KNC Accuracy']) 

counter = 1
while counter <= 20:
    df6 = pd.read_csv('data/Sigma/v6.'+str(counter)+'_data_dog_KM.csv')
    data = df6.values

    # Splitting the data
    X, y = data[:, :-1], data[:, -1]
    # using the train test split function
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=1)
    ### print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

    # Random Forest Classifier:
    model = RandomForestClassifier(random_state=1)
    model.fit(X_train, y_train)
    # make predictions
    yhat = model.predict(X_test)
    # evaluate predictions
    acc = accuracy_score(y_test, yhat)

    # K Nearest Neighbor Classification:
    knn = KNeighborsClassifier(n_neighbors=5)
    knn.fit(X_train, y_train)
    # make predictions
    y_pred = knn.predict(X_test)
    # evaluate predictions

    df.loc[counter]=[acc, metrics.accuracy_score(y_test, y_pred)]
    counter += 1

df

,RFC Accuracy,KNC Accuracy
1,0.722222,0.666667
2,0.808511,0.744681
3,0.773109,0.739496
4,0.811475,0.737705
5,0.795455,0.795455
6,0.805556,0.770833
7,0.739437,0.718310
8,0.756944,0.722222
9,0.760870,0.739130
10,0.819549,0.781955


In [26]:
df.to_csv('data/Sigma/Score_results.csv', index=False)